# **Evaluacion de Data set**
Tecnicatura en inteligencia artificial

Universidad Nacional de Hurlingham

Docente: Andrea Rey

Alumno : Nicolas Seivane

Librerias que voy a utilizar


In [1]:
import pandas as pd
!pip install scikit-learn seaborn matplotlib
import sklearn as sk
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    matthews_corrcoef,
    balanced_accuracy_score,
    log_loss,
    cohen_kappa_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import StratifiedKFold# .head()
# .info()
# .isnull().any()
# df.column.value_counts()
# .max()
# .min()
# .describe()


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'sklearn'

In [7]:
datos = pd.read_csv("C:\\Users\\User\\Documents\\Proyecto integrador\\heart\\heart.csv")

datos.drop_duplicates(inplace=True)
datos.dropna(inplace=True)
#datos.drop(columns=["CLASS"], inplace=True)
#datos['NSP'] = datos['NSP'].replace({1: 'Normal', 2: 'Sospechoso', 3: 'Patologico'})
indices = datos[datos['Cholesterol'] == 0].index
datos.drop(indices, inplace=True)
datos.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
datos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [4]:
datos.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [8]:
n_filas = len(datos)

for column in datos.columns:
    unicos = datos[column].unique()
    n_unicos = len(unicos)
    
    # Si es categórica (tipo object/category) o es numérica con pocos únicos
    if (datos[column].dtype == 'object') or (len(unicos) <= 10):
        print(f"\nColumna '{column}' (candidata categórica): {n_unicos} valores únicos")
        
        
        for unico in unicos:
                count = len(datos[datos[column] == unico])
                normalizado = count / n_filas
                print(f"   Hay {normalizado:.2%} codificado en {unico}")
    else:
        print(f"\nColumna '{column}' tiene media: {datos[column].mean():.4f}, valor maximo: {datos[column].max():.4f} y valor minimo: {datos[column].min():.4f}")


Columna 'Age' tiene media: 52.8820, valor maximo: 77.0000 y valor minimo: 28.0000

Columna 'Sex' (candidata categórica): 2 valores únicos
   Hay 75.60% codificado en M
   Hay 24.40% codificado en F

Columna 'ChestPainType' (candidata categórica): 4 valores únicos
   Hay 22.25% codificado en ATA
   Hay 22.65% codificado en NAP
   Hay 49.60% codificado en ASY
   Hay 5.50% codificado en TA

Columna 'RestingBP' tiene media: 133.0228, valor maximo: 200.0000 y valor minimo: 92.0000

Columna 'Cholesterol' tiene media: 244.6354, valor maximo: 603.0000 y valor minimo: 85.0000

Columna 'FastingBS' (candidata categórica): 2 valores únicos
   Hay 83.24% codificado en 0
   Hay 16.76% codificado en 1

Columna 'RestingECG' (candidata categórica): 3 valores únicos
   Hay 59.65% codificado en Normal
   Hay 16.76% codificado en ST
   Hay 23.59% codificado en LVH

Columna 'MaxHR' tiene media: 140.2265, valor maximo: 202.0000 y valor minimo: 69.0000

Columna 'ExerciseAngina' (candidata categórica): 2 val

In [ ]:
for column in datos.columns:
    if len(datos[column].unique()) <= 10 and datos[column].dtype == 'object':
        print(column, datos[column].unique())



In [ ]:
for column in datos.columns:
    if len(datos[column].unique()) <= 10 and datos[column].dtype == 'object':
        unicos = datos[column].unique()
        for i in range(len(unicos)):
            datos[column] = datos[column].replace({unicos[i]: i})
        print(column, datos[column].unique())

In [ ]:
for column in datos.columns:
    if len(datos[column].unique()) > 4:
        datos[column] = (datos[column] - datos[column].mean()) / datos[column].std()

In [ ]:
atributos = datos.columns[:-1]
concepto = datos.columns[-1]

datos.head()

In [ ]:
def validacion_cruzada(conjunto, atributos, concepto, k=5, random=False, agregar_unos=False, undersample=False, oversample=False):
  
  if undersample:
    cantidad_menor_concepto = conjunto[concepto].value_counts().min()
    conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_menor_concepto, random_state=42) for valor in conjunto[concepto].unique()])

  if oversample:
    cantidad_mayor_concepto = conjunto[concepto].value_counts().max()
    conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_mayor_concepto, replace=True, random_state=42) for valor in conjunto[concepto].unique()])

  if random: conjunto = conjunto.sample(frac=1, random_state=11).reset_index(drop=True)

  x = conjunto[atributos]
  if agregar_unos: x.insert(0, "Unos", 1)

  y = conjunto[concepto]

  resultados = []
  skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

  for train_index, test_index in skf.split(x, y):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        resultados.append((x_train, y_train, x_test, y_test))

  return resultados

from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier(
    n_estimators=100,      # número de árboles en el bosque
    criterion="gini",      # función de calidad de la división ("gini" o "entropy")
    max_depth=None,        # profundidad máxima de cada árbol
    min_samples_split=2,   # min muestras para dividir un nodo
    min_samples_leaf=1,    # min muestras en una hoja
    max_features="sqrt",   # nº de features a considerar en cada split ("sqrt", "log2", None)
    bootstrap=True,        # si usar bootstrap samples para entrenar cada árbol
    random_state=42,       # semilla
    n_jobs=-1              # usa todos los cores
)

from sklearn.naive_bayes import GaussianNB

GaussianNB(
    var_smoothing=1e-9     # suavizado para evitar divisiones por cero
)


from sklearn.svm import SVC

SVC(
    C=1.0,                 # penalización del error (mayor C = menos margen, más sobreajuste)
    kernel="rbf",          # kernel: "linear", "poly", "rbf", "sigmoid"
    degree=3,              # grado del polinomio (si kernel="poly")
    gamma="scale",         # controla la influencia de un solo ejemplo ("scale", "auto" o un valor numérico)
    probability=True,     # si calcular probabilidades (más lento)
    random_state=42
)

from sklearn.linear_model import LogisticRegression

LogisticRegression(
    penalty="l2",          # regularización: "l1", "l2", "elasticnet", None
    C=1.0,                 # inverso de la fuerza de regularización (menor C = más regularización)
    solver="lbfgs",        # optimizador ("lbfgs", "saga", "liblinear", "newton-cg")
    max_iter=1000,         # iteraciones máximas (importante en datasets grandes)
    multi_class="multinomial",    # "ovr" (uno-vs-rest) o "multinomial"
    random_state=42
)

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    matthews_corrcoef,
    balanced_accuracy_score,
    log_loss,
    cohen_kappa_score
)

    y_true_total.extend(y_test)
    y_pred_total.extend(y_pred)

print("Matriz de confusión total:\n", confusion_matrix(y_true_total, y_pred_total))

LogisticRegression(
    penalty="l2",          # regularización: "l1", "l2", "elasticnet", None
    C=1.0,                 # inverso de la fuerza de regularización (menor C = más regularización)
    solver="lbfgs",        # optimizador ("lbfgs", "saga", "liblinear", "newton-cg")
    max_iter=1000,         # iteraciones máximas (importante en datasets grandes)
    multi_class="multinomial",    # "ovr" (uno-vs-rest) o "multinomial"
    random_state=42
)

# *Regresion Logistica*

In [ ]:
grid = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l1","l2","elasticnet", None],  # 'l1' si usas solver='liblinear' o 'saga'
    "solver": ["lbfgs","saga", "newton-cg"],# según el penalty que uses, liblinear no soporta None, newton-cg no soporta l1 ni elasticnet
    "multi_class": ["ovr", "multinomial"] # si es multiclass
}
 
## cohen_kappa_score,     accuracy_score,    precision_score,     recall_score,    f1_score,    roc_auc_score,


#with open("regresion_logistica_resultados.txt", "w") as f:
with open("regresion_logistica_resultados_corazon.txt", "w") as f:
    f.write("Resultados de Regresion Logistica con validacion cruzada (5 folds):\n")
    
    for C in grid["C"]:

        for penalty in grid["penalty"]:
            for multi_class in grid["multi_class"]:
                for solver in grid["solver"]:
                    # chequeo de compatibilidad
                    if penalty == "l1" and solver not in ["liblinear", "saga"]:
                        continue
                    if penalty == "elasticnet" and solver != "saga":
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    scoring_funcs = {
                                "cohen_kappa": 0,
                                "accuracy": 0,
                                "precision": 0,
                                "recall": 0,
                                "f1": 0,
                                "roc_auc": 0
                            }
                    try:
                        inicio = time.time()
                        modelo = LogisticRegression(
                            C=C,
                            penalty=penalty,
                            solver=solver,
                            l1_ratio=0.5 if penalty=="elasticnet" else None,
                            multi_class=multi_class,
                            max_iter=1000,
                            random_state=42
                        )

                        folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)
                        
                        for x_train, y_train, x_test, y_test in folds:
                                modelo.fit(x_train, y_train)
                                pred = modelo.predict(x_test)
                                scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                probs = modelo.predict_proba(x_test)
                                if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                
                        fin = time.time()
                        tiempo_total = fin - inicio
                        for key in scoring_funcs:
                            scoring_funcs[key] /= len(folds) 
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                    except Exception as e:
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Regresion Logistica con validacion cruzada (5 folds) undersample:\n")
    
    for C in grid["C"]:

        for penalty in grid["penalty"]:
            for solver in grid["solver"]:
                for multi_class in grid["multi_class"]:
                    # chequeo de compatibilidad
                    if penalty == "l1" and solver not in ["liblinear", "saga"]:
                        continue
                    if penalty == "elasticnet" and solver != "saga":
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    scoring_funcs = {
                                "cohen_kappa": 0,
                                "accuracy": 0,
                                "precision": 0,
                                "recall": 0,
                                "f1": 0,
                                "roc_auc": 0
                            }
                    try:
                        tiempo_inicio = time.time()
                        modelo = LogisticRegression(
                            C=C,
                            penalty=penalty,
                            solver=solver,
                            l1_ratio=0.5 if penalty=="elasticnet" else None,
                            multi_class=multi_class,
                            max_iter=1000,
                            random_state=42
                        )

                        folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                        for x_train, y_train, x_test, y_test in folds:
                                modelo.fit(x_train, y_train)
                                pred = modelo.predict(x_test)
                                scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                probs = modelo.predict_proba(x_test)
                                if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")


                        fin = time.time()
                        tiempo_total = fin - tiempo_inicio
                        for key in scoring_funcs:
                            scoring_funcs[key] /= len(folds)  # Promediar las métricas

                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                    except Exception as e:
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> ERROR: {str(e)}\n")

RandomForestClassifier(
    n_estimators=100,      # número de árboles en el bosque
    criterion="gini",      # función de calidad de la división ("gini" o "entropy")
    max_depth=None,        # profundidad máxima de cada árbol
    min_samples_split=2,   # min muestras para dividir un nodo
    min_samples_leaf=1,    # min muestras en una hoja
    max_features="sqrt",   # nº de features a considerar en cada split ("sqrt", "log2", None)
    bootstrap=True,        # si usar bootstrap samples para entrenar cada árbol
    random_state=42,       # semilla
    n_jobs=-1              # usa todos los cores
)


# *Random Forest*

In [ ]:
grid = {
    "n_estimators": [50, 100, 200, 500, 1000],
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 3, 5, 7, 9],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [None, "sqrt", "log2"]
}



#with open("Random_forest.txt", "w") as f:
with open("Random_forest_corazon.txt", "w") as f:
    f.write("Resultados de Random Forest con validacion cruzada (5 folds):\n")

    for criterion in grid["criterion"]:
        for max_depth in grid["max_depth"]:
            for min_samples_split in grid["min_samples_split"]:
                for min_samples_leaf in grid["min_samples_leaf"]:
                    for max_features in grid["max_features"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = RandomForestClassifier(
                                criterion=criterion,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                max_features=max_features,
                                random_state=42
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} "
                                    f"--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")
                        except Exception as e:
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Random Forest con validacion cruzada (5 folds) undersample:\n")
    
    for criterion in grid["criterion"]:
        for max_depth in grid["max_depth"]:
            for min_samples_split in grid["min_samples_split"]:
                for min_samples_leaf in grid["min_samples_leaf"]:
                    for max_features in grid["max_features"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = RandomForestClassifier(
                                criterion=criterion,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                max_features=max_features,
                                random_state=42
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} "
                                    f"--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")
                        except Exception as e:
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} --> ERROR: {str(e)}\n")

GaussianNB(
    var_smoothing=1e-9     # suavizado para evitar divisiones por cero
)


# *Naive Bayes*

In [ ]:
grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]  
}

#with open("Naiva_Bayes.txt", "w") as f:
with open("Naiva_Bayes_corazon.txt", "w") as f:
    f.write("Resultados de Naive Bayes con validacion cruzada (5 folds):\n")

    for var_smoothing in grid["var_smoothing"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = GaussianNB(var_smoothing=var_smoothing)

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\n var_smoothing={var_smoothing}--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\n var_smoothing={var_smoothing}--> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Naive Bayes con validacion cruzada (5 folds) undersample:\n")

    for var_smoothing in grid["var_smoothing"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = GaussianNB(var_smoothing=var_smoothing)
                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\n var_smoothing={var_smoothing}--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\n var_smoothing={var_smoothing}--> ERROR: {str(e)}\n")

SVC(
    C=1.0,                 # penalización del error (mayor C = menos margen, más sobreajuste)
    kernel="rbf",          # kernel: "linear", "poly", "rbf", "sigmoid"
    degree=3,              # grado del polinomio (si kernel="poly")
    gamma="scale",         # controla la influencia de un solo ejemplo ("scale", "auto" o un valor numérico)
    probability=True,     # si calcular probabilidades (más lento)
    random_state=42
)


# *SVM*

In [ ]:
grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 15, 20, 25],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "gamma": ["scale", "auto", 0.001, 0.01, 0.1, 1],
    "degree": [2,3,4,5,6,7,8,9,10]
}

#with open("SVM.txt", "w") as f:
with open("SVM_corazon.txt", "w") as f:
    f.write("Resultados de SVM con validacion cruzada (5 folds):\n")

    for C in grid["C"]:
        for kernel in grid["kernel"]:
            for gamma in grid["gamma"]:
                degrees_to_use = grid["degree"] if kernel == "poly" else [3]
                for degree in degrees_to_use:
                        scoring_funcs = {
                                "cohen_kappa": 0,
                                 "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = SVC(
                                  C=C,
                                    kernel=kernel,
                                    gamma=gamma,
                                    degree=degree,
                                    max_iter=1000,
                                    random_state=42,
                                    probability=True
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)

                                    if len(set(y_test)) == 2:
                                        # binaria
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\nC={C}, kernel={kernel}, gamma={gamma}, degree={degree} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\nC={C}, kernel={kernel}, gamma={gamma}, degree={degree} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de SVM con validacion cruzada (5 folds) undersample:\n")

    for C in grid["C"]:
        for kernel in grid["kernel"]:
            for gamma in grid["gamma"]:
                if kernel != "poly" and "degree" in grid:
                    continue
                for degree in grid["degree"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = SVC(
                                  C=C,
                                    kernel=kernel,
                                    gamma=gamma,
                                    degree=degree if kernel=="poly" else None,
                                    max_iter=1000,
                                    random_state=42,
                                    probability=True
                            )
                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)

                                    if len(set(y_test)) == 2:
                                        # binaria
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\nC={C}, C={C}, gamma={gamma}, degree={degree} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\nC={C}, C={C}, gamma={gamma}, degree={degree} --> ERROR: {str(e)}\n")

In [ ]:
from sklearn.inspection import permutation_importance

best_lr_model = LogisticRegression(C=1.0, penalty='l1', solver='saga', multi_class='ovr', max_iter=1000, random_state=42)
best_svm_model = SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42, probability=True)
best_gnb_model = GaussianNB(var_smoothing=1e-09)
best_rf_model = RandomForestClassifier(criterion='entropy', max_depth=7, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', random_state=42)


best_lr_model.fit(datos[atributos], datos[concepto])
best_svm_model.fit(datos[atributos], datos[concepto])
best_gnb_model.fit(datos[atributos], datos[concepto])
best_rf_model.fit(datos[atributos], datos[concepto])

with open("Importancia_corazon.txt", "w") as f:
    f.write("La importancia de las features:\n")

    f.write("Random Forest Importancia atributos:")
    feature_importances_rf = pd.Series(best_rf_model.feature_importances_, index=atributos)
    f.write(str(feature_importances_rf.sort_values(ascending=False)))
    f.write("-" * 30)


    f.write("Logistic Regression Importancia Permutacion:")
    result_lr = permutation_importance(best_lr_model, datos[atributos], datos[concepto], n_repeats=10, random_state=42, n_jobs=-1)
    feature_importances_lr = pd.Series(result_lr.importances_mean, index=atributos)
    f.write(str(feature_importances_lr.sort_values(ascending=False)))
    f.write("-" * 30)

    f.write("SVM Importancia Permutacion:")
    result_svm = permutation_importance(best_svm_model, datos[atributos], datos[concepto], n_repeats=10, random_state=42, n_jobs=-1)
    feature_importances_svm = pd.Series(result_svm.importances_mean, index=atributos)
    f.write(str(feature_importances_svm.sort_values(ascending=False)))
    f.write("-" * 30)

    f.write("Gaussiano Naive Bayes Importancia Permutacion:")
    result_gnb = permutation_importance(best_gnb_model, datos[atributos], datos[concepto], n_repeats=10, random_state=42, n_jobs=-1)
    feature_importances_gnb = pd.Series(result_gnb.importances_mean, index=atributos)
    f.write(str(feature_importances_gnb.sort_values(ascending=False)))
    f.write("-" * 30)


In [ ]:
from sklearn.inspection import permutation_importance

best_lr_model = LogisticRegression(C=1.0, penalty='l1', solver='saga', multi_class='ovr', max_iter=1000, random_state=42)
best_svm_model = SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42, probability=True)
best_gnb_model = GaussianNB(var_smoothing=1e-09)
best_rf_model = RandomForestClassifier(criterion='entropy', max_depth=7, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', random_state=42)


best_lr_model.fit(datos[atributos], datos[concepto])
best_svm_model.fit(datos[atributos], datos[concepto])
best_gnb_model.fit(datos[atributos], datos[concepto])
best_rf_model.fit(datos[atributos], datos[concepto])

feature_importances_rf = pd.Series(best_rf_model.feature_importances_, index=atributos)

result_lr = permutation_importance(best_lr_model, datos[atributos], datos[concepto], n_repeats=10, random_state=42, n_jobs=-1)
feature_importances_lr = pd.Series(result_lr.importances_mean, index=atributos)

result_svm = permutation_importance(best_svm_model, datos[atributos], datos[concepto], n_repeats=10, random_state=42, n_jobs=-1)
feature_importances_svm = pd.Series(result_svm.importances_mean, index=atributos)

result_gnb = permutation_importance(best_gnb_model, datos[atributos], datos[concepto], n_repeats=10, random_state=42, n_jobs=-1)
feature_importances_gnb = pd.Series(result_gnb.importances_mean, index=atributos)



print(feature_importances_lr.index)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, cohen_kappa_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import time


concepto = datos.columns[-1]

atributos = []

# The original code was trying to iterate over a pandas Series as if it were a list of strings.
# We need to iterate over the index of the Series to get the attribute names.
with open("Logistic_Regression_resultados_Corazon_atributos.txt", "w") as f:
    f.write("Resultados de Regresion Logistica con validacion cruzada (5 folds) con seleccion de atributos:\n")
    f1 = []
    auc = []
    acc = []
    for atributo in feature_importances_lr.index:
      atributos += [atributo]

      scoring_funcs = {
                  "cohen_kappa": 0,
                  "accuracy": 0,
                  "precision": 0,
                  "recall": 0,
                  "f1": 0,
                  "roc_auc": 0
              }
      inicio = time.time()

      folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

      for x_train, y_train, x_test, y_test in folds:
                  best_lr_model.fit(x_train, y_train)
                  pred = best_lr_model.predict(x_test)
                  scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                  scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                  scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                  scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                  scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                  probs = best_lr_model.predict_proba(x_test)
                  if len(set(y_test)) == 2:
                      # binaria
                      scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                  else:
                      # multiclase
                      scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

      fin = time.time()
      tiempo_total = fin - inicio
      for key in scoring_funcs:
              scoring_funcs[key] /= len(folds)

      print(f"  Average Metrics with attributes {atributos}: {scoring_funcs}")
      print(f"  Total Time: {tiempo_total:.2f} seconds")
      f.write(f"  Average Metrics with attributes {atributos}: {scoring_funcs}, Total Time: {tiempo_total:.2f} seconds\n")
      f1.append(scoring_funcs['f1'])
      f.write(f"F1 = {f1}\n")
      auc.append(scoring_funcs['roc_auc'])
      f.write(f"AUC = {auc}\n")
      acc.append(scoring_funcs['accuracy'])
      f.write(f"Accuracy = {acc}\n")
      f.write(f"Total Time: {tiempo_total:.2f} seconds\n")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, cohen_kappa_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import time

concepto = datos.columns[-1]

atributos_svm = []

# Using feature importances from SVM permutation importance
with open("SVM_resultados_Corazon_atributos.txt", "w") as f:
    f.write("Resultados de SVM con validacion cruzada (5 folds) con seleccion de atributos:\n")
    f1 = []
    auc = []
    acc = []
    for atributo in feature_importances_svm.sort_values(ascending=False).index:
      atributos_svm += [atributo]

      scoring_funcs = {
                  "cohen_kappa": 0,
                  "accuracy": 0,
                  "precision": 0,
                  "recall": 0,
                  "f1": 0,
                  "roc_auc": 0
              }
      inicio = time.time()

      folds = validacion_cruzada(datos, atributos_svm, concepto, k=5, random=True)

      for x_train, y_train, x_test, y_test in folds:
              best_svm_model.fit(x_train, y_train)
              pred = best_svm_model.predict(x_test)
              scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
              scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
              scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
              scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
              scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
              probs = best_svm_model.predict_proba(x_test)
              if len(set(y_test)) == 2:
                  # binaria
                  scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
              else:
                  # multiclase
                  scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

      fin = time.time()
      tiempo_total = fin - inicio
      for key in scoring_funcs:
              scoring_funcs[key] /= len(folds)

      print(f"  Average Metrics with attributes {atributos_svm}: {scoring_funcs}")
      print(f"  Total Time: {tiempo_total:.2f} seconds")
      f.write(f"  Average Metrics with attributes {atributos_svm}: {scoring_funcs}, Total Time: {tiempo_total:.2f} seconds\n")
      f1.append(scoring_funcs['f1'])
      f.write(f"F1 = {f1}\n")
      auc.append(scoring_funcs['roc_auc'])
      f.write(f"AUC = {auc}\n")
      acc.append(scoring_funcs['accuracy'])
      f.write(f"Accuracy = {acc}\n")
      f.write(f"Total Time: {tiempo_total:.2f} seconds\n")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, cohen_kappa_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import time

concepto = datos.columns[-1]

atributos_gb = [] # Changed variable name to reflect Random Forest

# Using feature importances from Random Forest feature importance
with open("Gauss_Bayes_Corazon_atributos.txt", "w") as f: # Added file writing
    f.write("Resultados de Naive Bayes con validacion cruzada (5 folds) con seleccion de atributos:\n") # Added header
    f1 = []
    auc = []
    acc = []
    for atributo in feature_importances_gnb.sort_values(ascending=False).index:
      atributos_gb += [atributo] # Changed variable name to reflect Random Forest

      scoring_funcs = {
                  "cohen_kappa": 0,
                  "accuracy": 0,
                  "precision": 0,
                  "recall": 0,
                  "f1": 0,
                  "roc_auc": 0
              }
      inicio = time.time()

      # Use datos_multinomial for MultinomialNB
      folds = validacion_cruzada(datos, atributos_gb, concepto, k=5, random=True) # Changed datos_multinomial to datos and atributos_gnb to atributos_rf

      for x_train, y_train, x_test, y_test in folds:
                  best_gnb_model.fit(x_train, y_train)
                  pred = best_gnb_model.predict(x_test)
                  scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                  scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                  scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                  scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                  scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                  probs = best_gnb_model.predict_proba(x_test)
                  if len(set(y_test)) == 2:
                      # binaria
                      scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                  else:
                      # multiclase
                      scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

      fin = time.time()
      tiempo_total = fin - inicio
      for key in scoring_funcs:
              scoring_funcs[key] /= len(folds)

      print(f"  Average Metrics with attributes {atributos_gb}: {scoring_funcs}")
      print(f"  Total Time: {tiempo_total:.2f} seconds")
      f.write(f"  Average Metrics with attributes {atributos_gb}: {scoring_funcs}, Total Time: {tiempo_total:.2f} seconds\n") # Added writing to file
      f1.append(scoring_funcs['f1'])
      f.write(f"F1 = {f1}\n")
      auc.append(scoring_funcs['roc_auc'])
      f.write(f"AUC = {auc}\n")
      acc.append(scoring_funcs['accuracy'])
      f.write(f"Accuracy = {acc}\n")
      f.write(f"Total Time: {tiempo_total:.2f} seconds\n")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, cohen_kappa_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import time

concepto = datos.columns[-1]

atributos_rf = [] # Changed variable name to reflect Random Forest

# Using feature importances from Random Forest feature importance
with open("Random_Forest_resultados_Corazon_atributos.txt", "w") as f: # Added file writing
    f.write("Resultados de Random Forest con validacion cruzada (5 folds) con seleccion de atributos:\n") # Added header
    f1 = []
    auc = []
    acc = []
    for atributo in feature_importances_rf.sort_values(ascending=False).index:
      atributos_rf += [atributo] # Changed variable name to reflect Random Forest

      scoring_funcs = {
                  "cohen_kappa": 0,
                  "accuracy": 0,
                  "precision": 0,
                  "recall": 0,
                  "f1": 0,
                  "roc_auc": 0
              }
      inicio = time.time()

      # Use datos_multinomial for MultinomialNB
      folds = validacion_cruzada(datos, atributos_rf, concepto, k=5, random=True) # Changed datos_multinomial to datos and atributos_gnb to atributos_rf

      for x_train, y_train, x_test, y_test in folds:
                  best_rf_model.fit(x_train, y_train)
                  pred = best_rf_model.predict(x_test)
                  scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                  scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                  scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                  scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                  scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                  probs = best_rf_model.predict_proba(x_test)
                  if len(set(y_test)) == 2:
                      # binaria
                      scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                  else:
                      # multiclase
                      scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

      fin = time.time()
      tiempo_total = fin - inicio
      for key in scoring_funcs:
              scoring_funcs[key] /= len(folds)

      print(f"  Average Metrics with attributes {atributos_rf}: {scoring_funcs}")
      print(f"  Total Time: {tiempo_total:.2f} seconds")
      f.write(f"  Average Metrics with attributes {atributos_rf}: {scoring_funcs}, Total Time: {tiempo_total:.2f} seconds\n") # Added writing to file
      f1.append(scoring_funcs['f1'])
      f.write(f"F1 = {f1}\n")
      auc.append(scoring_funcs['roc_auc'])
      f.write(f"AUC = {auc}\n")
      acc.append(scoring_funcs['accuracy'])
      f.write(f"Accuracy = {acc}\n")
      f.write(f"Total Time: {tiempo_total:.2f} seconds\n")